# Install Necessary Libraries



In [ ]:
!pip install flask flask-cors optuna xgboost lightgbm catboost tensorflow plotly dask[dataframe]


INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.2/242.2 kB 1.8 MB/s eta 0:00:00


# Generation of Synthetic Data

This code imports various Python libraries for data analysis, machine learning, and web development. It also defines constants related to battery assembly, including stages, cell types, welding methods, defect types, and parameter thresholds.

In [ ]:
import numpy as np
import pandas as pd
import optuna
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, IsolationForest
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.metrics import confusion_matrix, mean_squared_error, r2_score, accuracy_score
from scipy import stats
import plotly.express as px
import plotly.graph_objs as go
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate
from tensorflow.keras.callbacks import EarlyStopping

from flask import Flask, request, jsonify
from flask_cors import CORS
import json
from google.colab.output import eval_js
import random

# Constants
ASSEMBLY_STAGES = ['Pre_Assembly', 'Insulation_Tensioning', 'Electrical_Contacting', 'Slave_Board_Mounting', 'Housing_Cover_Mounting']
CELL_TYPES = ['Prismatic', 'Pouch', 'Cylindrical']
WELDING_METHODS = ['Laser', 'Ultrasonic', 'Resistance']
DEFECT_TYPES = ['No_Defect', 'Minor_Defect', 'Major_Defect', 'Critical_Defect']
THRESHOLDS = {
    'Cell_Voltage': (3.2, 4.2),
    'Cell_Impedance': (0.5, 2.0),
    'Cell_Capacity': (2000, 5000),  # mAh
    'Compression_Force': (500, 2000),  # N
    'Welding_Current': (100, 300),  # A
    'Welding_Time': (0.1, 1.0),  # s
    'Torque': (5, 15),  # Nm
    'Assembly_Time': (10, 30),  # min
    'Leakage_Rate': (0, 0.1),  # cm³/min
}

# EV Module Assembly Analyzer Class

This code defines an `EVModuleAssemblyAnalyzer` class for analyzing EV module assembly processes. It includes methods for data generation, model training, anomaly detection, result visualization, and process optimization using various machine learning techniques. The class also provides functionality for running what-if scenarios and detecting errors in the assembly process.

In [ ]:
class EVModuleAssemblyAnalyzer:
    def __init__(self):
        self.data = None
        self.models = {}
        self.scaler = None
        self.feature_names = None
        self.label_encoder = None

    def generate_data(self, num_samples=5000):  # Reduced from 50000 to 5000
        np.random.seed(42)
        self.data = pd.DataFrame({
            'Assembly_Stage': np.random.choice(ASSEMBLY_STAGES, num_samples),
            'Cell_Type': np.random.choice(CELL_TYPES, num_samples),
            'Welding_Method': np.random.choice(WELDING_METHODS, num_samples),
            'Cell_Voltage': np.random.normal(3.7, 0.2, num_samples),
            'Cell_Impedance': np.random.normal(1.0, 0.3, num_samples),
            'Cell_Capacity': np.random.normal(3500, 500, num_samples),
            'Compression_Force': np.random.normal(1000, 200, num_samples),
            'Welding_Current': np.random.normal(200, 30, num_samples),
            'Welding_Time': np.random.normal(0.5, 0.1, num_samples),
            'Torque': np.random.normal(10, 1, num_samples),
            'Assembly_Time': np.random.normal(20, 3, num_samples),
            'Leakage_Rate': np.random.exponential(0.05, num_samples),
            'Operator_ID': np.random.randint(1, 10, num_samples),  # Reduced from 50 to 10
            'Equipment_ID': np.random.randint(1, 5, num_samples),  # Reduced from 20 to 5
            'Raw_Material_Batch': np.random.randint(1, 20, num_samples),  # Reduced from 100 to 20
            'Room_Temperature': np.random.normal(22, 2, num_samples),
            'Room_Humidity': np.random.normal(50, 5, num_samples),
            'Shift': np.random.choice(['Morning', 'Afternoon', 'Night'], num_samples),
            'Production_Rate': np.random.uniform(50, 100, num_samples),
            'Defect_Type': np.random.choice(DEFECT_TYPES, num_samples, p=[0.7, 0.15, 0.1, 0.05]),
            'Efficiency_Score': np.random.uniform(80, 100, num_samples)
        })

    def optimize_hyperparameters(self, model_type, X, y_defect, y_efficiency):
        def objective(trial):
            if model_type in ['rf', 'xgb', 'lgbm', 'catboost']:
                if model_type == 'rf':
                    params = {
                        'n_estimators': trial.suggest_int('n_estimators', 50, 200),  # Reduced from 100-1000 to 50-200
                        'max_depth': trial.suggest_int('max_depth', 3, 10),  # Reduced from 5-30 to 3-10
                        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),  # Reduced from 2-20 to 2-10
                        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5)  # Reduced from 1-10 to 1-5
                    }
                    model_defect = RandomForestClassifier(**params, random_state=42)
                    model_efficiency = RandomForestRegressor(**params, random_state=42)
                elif model_type == 'xgb':
                    params = {
                        'max_depth': trial.suggest_int('max_depth', 3, 8),  # Reduced from 3-10 to 3-8
                        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.5),  # Reduced upper bound from 1.0 to 0.5
                        'n_estimators': trial.suggest_int('n_estimators', 50, 200),  # Reduced from 100-1000 to 50-200
                        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),  # Reduced from 1-10 to 1-5
                        'subsample': trial.suggest_uniform('subsample', 0.6, 1.0)
                    }
                    model_defect = XGBClassifier(**params, random_state=42)
                    model_efficiency = XGBRegressor(**params, random_state=42)
                elif model_type == 'lgbm':
                    params = {
                        'num_leaves': trial.suggest_int('num_leaves', 20, 50),  # Reduced from 20-100 to 20-50
                        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.5),  # Reduced upper bound from 1.0 to 0.5
                        'n_estimators': trial.suggest_int('n_estimators', 50, 200),  # Reduced from 100-1000 to 50-200
                        'min_child_samples': trial.suggest_int('min_child_samples', 5, 50),  # Reduced from 5-100 to 5-50
                        'subsample': trial.suggest_uniform('subsample', 0.6, 1.0)
                    }
                    model_defect = LGBMClassifier(**params, random_state=42)
                    model_efficiency = LGBMRegressor(**params, random_state=42)
                else:  # catboost
                    params = {
                        'depth': trial.suggest_int('depth', 4, 8),  # Reduced from 4-10 to 4-8
                        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.5),  # Reduced upper bound from 1.0 to 0.5
                        'iterations': trial.suggest_int('iterations', 50, 200)  # Reduced from 100-1000 to 50-200
                    }
                    model_defect = CatBoostClassifier(**params, random_state=42, verbose=0)
                    model_efficiency = CatBoostRegressor(**params, random_state=42, verbose=0)

                defect_score = cross_val_score(model_defect, X, y_defect, cv=3, scoring='accuracy').mean()  # Reduced from cv=5 to cv=3
                efficiency_score = -cross_val_score(model_efficiency, X, y_efficiency, cv=3, scoring='neg_mean_squared_error').mean()  # Reduced from cv=5 to cv=3

                return defect_score - efficiency_score  # Optimize both objectives

            else:
                raise ValueError("Unsupported model type")

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=20)  # Reduced from 100 to 20

        return study.best_params

    def preprocess_data(self):
        df_encoded = pd.get_dummies(self.data, columns=['Assembly_Stage', 'Cell_Type', 'Welding_Method', 'Shift'])
        X = df_encoded.drop(['Defect_Type', 'Efficiency_Score'], axis=1)
        y_defect = df_encoded['Defect_Type']
        y_efficiency = df_encoded['Efficiency_Score']

        # Encode the defect labels
        self.label_encoder = LabelEncoder()
        y_defect_encoded = self.label_encoder.fit_transform(y_defect)

        X_train, X_test, y_defect_train, y_defect_test, y_efficiency_train, y_efficiency_test = train_test_split(
            X, y_defect_encoded, y_efficiency, test_size=0.2, random_state=42)

        self.scaler = StandardScaler()
        X_train_scaled = self.scaler.fit_transform(X_train)
        X_test_scaled = self.scaler.transform(X_test)

        self.feature_names = X.columns
        return X_train_scaled, X_test_scaled, y_defect_train, y_defect_test, y_efficiency_train, y_efficiency_test


    def build_multi_input_nn(self, input_dim):
        input_layer = Input(shape=(input_dim,))
        hidden1 = Dense(64, activation='relu')(input_layer)
        hidden2 = Dense(32, activation='relu')(hidden1)

        defect_output = Dense(len(DEFECT_TYPES), activation='softmax', name='defect_output')(hidden2)
        efficiency_output = Dense(1, name='efficiency_output')(hidden2)

        model = Model(inputs=input_layer, outputs=[defect_output, efficiency_output])
        model.compile(optimizer='adam',
                      loss={'defect_output': 'categorical_crossentropy', 'efficiency_output': 'mse'},
                      metrics={'defect_output': 'accuracy', 'efficiency_output': 'mse'})
        return model


    def train_models(self, X_train, y_defect_train, y_efficiency_train):
        for model_type in ['rf', 'xgb', 'lgbm', 'catboost']:
            params = self.optimize_hyperparameters(model_type, X_train, y_defect_train, y_efficiency_train)

            if model_type == 'rf':
                self.models[f'{model_type}_defect'] = RandomForestClassifier(**params, random_state=42)
                self.models[f'{model_type}_efficiency'] = RandomForestRegressor(**params, random_state=42)
            elif model_type == 'xgb':
                self.models[f'{model_type}_defect'] = XGBClassifier(**params, random_state=42)
                self.models[f'{model_type}_efficiency'] = XGBRegressor(**params, random_state=42)
            elif model_type == 'lgbm':
                self.models[f'{model_type}_defect'] = LGBMClassifier(**params, random_state=42)
                self.models[f'{model_type}_efficiency'] = LGBMRegressor(**params, random_state=42)
            elif model_type == 'catboost':
                self.models[f'{model_type}_defect'] = CatBoostClassifier(**params, random_state=42, verbose=0)
                self.models[f'{model_type}_efficiency'] = CatBoostRegressor(**params, random_state=42, verbose=0)

            self.models[f'{model_type}_defect'].fit(X_train, y_defect_train)
            self.models[f'{model_type}_efficiency'].fit(X_train, y_efficiency_train)

        self.models['nn'] = self.build_multi_input_nn(X_train.shape[1])
        early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
        self.models['nn'].fit(X_train,
                              {'defect_output': pd.get_dummies(y_defect_train), 'efficiency_output': y_efficiency_train},
                              epochs=30, batch_size=32, validation_split=0.2,
                              callbacks=[early_stopping], verbose=0)


    def ensemble_predict(self, X):
        defect_predictions = []
        efficiency_predictions = []

        for model_type in ['rf', 'xgb', 'lgbm', 'catboost']:
            defect_pred = self.models[f'{model_type}_defect'].predict(X)
            efficiency_pred = self.models[f'{model_type}_efficiency'].predict(X)
            defect_predictions.append(defect_pred)
            efficiency_predictions.append(efficiency_pred)

        nn_defect_pred, nn_efficiency_pred = self.models['nn'].predict(X)
        nn_defect_pred = np.argmax(nn_defect_pred, axis=1)
        defect_predictions.append(nn_defect_pred)
        efficiency_predictions.append(nn_efficiency_pred.flatten())

        ensemble_defect_pred = stats.mode(defect_predictions, axis=0)[0].flatten()
        ensemble_efficiency_pred = np.mean(efficiency_predictions, axis=0)

        return ensemble_defect_pred, ensemble_efficiency_pred

    def detect_anomalies(self, data, contamination=0.01):
        clf = IsolationForest(contamination=contamination, random_state=42)
        data_numeric = data.select_dtypes(include=np.number)
        anomalies = clf.fit_predict(data_numeric) == -1
        return data[anomalies]



    def ensemble_predict(self, X):
        defect_predictions = []
        efficiency_predictions = []

        for model_type in ['rf', 'xgb', 'lgbm', 'catboost']:
            defect_pred = self.models[f'{model_type}_defect'].predict(X)
            efficiency_pred = self.models[f'{model_type}_efficiency'].predict(X)
            defect_predictions.append(defect_pred)
            efficiency_predictions.append(efficiency_pred)

        nn_defect_pred, nn_efficiency_pred = self.models['nn'].predict(X)
        nn_defect_pred = np.argmax(nn_defect_pred, axis=1)
        defect_predictions.append(nn_defect_pred)
        efficiency_predictions.append(nn_efficiency_pred.flatten())

        # Ensure all predictions have the same shape
        defect_predictions = [pred.reshape(-1, 1) if pred.ndim == 1 else pred for pred in defect_predictions]
        efficiency_predictions = [pred.reshape(-1, 1) if pred.ndim == 1 else pred for pred in efficiency_predictions]

        ensemble_defect_pred = stats.mode(defect_predictions, axis=0)[0].flatten()
        ensemble_efficiency_pred = np.mean(efficiency_predictions, axis=0).flatten()

        return ensemble_defect_pred, ensemble_efficiency_pred

    def analyze_and_recommend(self, X):
        defect_predictions, efficiency_predictions = self.ensemble_predict(X)
        feature_importance = self.models['rf_defect'].feature_importances_

        sorted_idx = np.argsort(feature_importance)
        top_features = self.feature_names[sorted_idx[-10:]]

        recommendations = []
        for feature in top_features:
            if 'Assembly_Stage' in feature:
                stage = feature.split('_')[-1]
                recommendations.append(f"High defect risk detected in {stage} stage. Review process controls.")
            elif 'Cell_Type' in feature:
                cell_type = feature.split('_')[-1]
                recommendations.append(f"{cell_type} cells significantly impact defects. Enhance quality control.")
            elif 'Welding_Method' in feature:
                method = feature.split('_')[-1]
                recommendations.append(f"Optimize {method} welding parameters. Implement real-time monitoring.")
            else:
                recommendations.append(f"Optimize {feature} control. Implement adaptive process control.")

        return recommendations, feature_importance, defect_predictions, efficiency_predictions

    def visualize_results(self, feature_importance, y_true_defect, y_pred_defect, y_true_efficiency, y_pred_efficiency):
        # Feature importance plot
        sorted_idx = np.argsort(feature_importance)
        top_features = self.feature_names[sorted_idx[-10:]]
        top_importance = feature_importance[sorted_idx[-10:]]
        fig = px.bar(x=top_importance, y=top_features, orientation='h', title='Top 10 Feature Importance')
        fig.show()

        # Confusion matrix for defect prediction
        cm = confusion_matrix(y_true_defect, y_pred_defect)
        defect_labels = self.label_encoder.classes_
        fig = px.imshow(cm, labels=dict(x="Predicted", y="Actual"), x=defect_labels, y=defect_labels,
                        title='Confusion Matrix - Defect Prediction', color_continuous_scale='Blues')
        fig.update_xaxes(side="top")
        fig.show()

        # Efficiency prediction scatter plot
        fig = px.scatter(x=y_true_efficiency, y=y_pred_efficiency,
                         labels={'x': 'Actual Efficiency', 'y': 'Predicted Efficiency'},
                         title='Efficiency Prediction Performance')
        fig.add_trace(go.Scatter(x=[min(y_true_efficiency), max(y_true_efficiency)],
                                 y=[min(y_true_efficiency), max(y_true_efficiency)],
                                 mode='lines', name='Perfect Prediction'))
        fig.show()

        # Print efficiency prediction metrics
        mse = mean_squared_error(y_true_efficiency, y_pred_efficiency)
        r2 = r2_score(y_true_efficiency, y_pred_efficiency)
        print(f"Efficiency Prediction - MSE: {mse:.4f}, R2 Score: {r2:.4f}")

    def what_if_scenario(self, scenario_changes):
        """
        Run a what-if scenario by modifying specific features and predicting the outcome.

        :param scenario_changes: dict, keys are feature names and values are the new values
        :return: dict with predicted defect probability and efficiency score
        """
        # Create a copy of the last row in the dataset
        base_scenario = self.data.iloc[-1:].copy()

        # Apply the changes
        for feature, value in scenario_changes.items():
            if feature in base_scenario.columns:
                base_scenario[feature] = value
            else:
                print(f"Warning: Feature '{feature}' not found in the dataset.")

        # Preprocess the scenario data
        scenario_encoded = pd.get_dummies(base_scenario, columns=['Assembly_Stage', 'Cell_Type', 'Welding_Method', 'Shift'])
        scenario_encoded = scenario_encoded.reindex(columns=self.feature_names, fill_value=0)
        scenario_scaled = self.scaler.transform(scenario_encoded)

        # Make predictions
        defect_prob, efficiency_score = self.ensemble_predict(scenario_scaled)

        # Handle both single-value and array predictions
        if np.isscalar(defect_prob):
            defect_prob = [defect_prob]
        if np.isscalar(efficiency_score):
            efficiency_score = [efficiency_score]

        # Ensure defect_prob has the same length as DEFECT_TYPES
        if len(defect_prob) < len(DEFECT_TYPES):
            defect_prob = np.pad(defect_prob, (0, len(DEFECT_TYPES) - len(defect_prob)), 'constant')
        elif len(defect_prob) > len(DEFECT_TYPES):
            defect_prob = defect_prob[:len(DEFECT_TYPES)]

        return {
            'defect_probability': {DEFECT_TYPES[i]: defect_prob[i] for i in range(len(DEFECT_TYPES))},
            'efficiency_score': efficiency_score[0]
        }

    def optimize_process(self, target_feature, constraints=None):
        def objective(trial):
            scenario = {target_feature: trial.suggest_uniform(target_feature,
                                                              THRESHOLDS[target_feature][0],
                                                              THRESHOLDS[target_feature][1])}
            if constraints:
                scenario.update(constraints)

            result = self.what_if_scenario(scenario)
            defect_score = 1 - result['defect_probability']['No_Defect']
            efficiency_score = result['efficiency_score']

            return efficiency_score - 100 * defect_score  # Penalize defects heavily

        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=20)  # Reduced from 100 to 20

        optimal_scenario = {target_feature: study.best_params[target_feature]}
        if constraints:
            optimal_scenario.update(constraints)

        optimal_result = self.what_if_scenario(optimal_scenario)

        return {
            'optimal_value': study.best_params[target_feature],
            'predicted_outcome': optimal_result
        }



    def detect_errors_and_suggest(self):
        errors = []
        for feature, (low, high) in THRESHOLDS.items():
            if feature in self.data.columns:
                out_of_range = self.data[(self.data[feature] < low) | (self.data[feature] > high)]
                if not out_of_range.empty:
                    errors.append(f"{feature} out of range in {len(out_of_range)} samples.")
                    errors.append(f"  - Check {feature.lower()} control systems and process parameters.")

        # Check for correlations specific to EV module assembly
        if 'Welding_Current' in self.data.columns and 'Welding_Time' in self.data.columns:
            welding_energy = self.data['Welding_Current'] * self.data['Welding_Time']
            if (welding_energy < 20).any() or (welding_energy > 300).any():
                errors.append("Inconsistent welding energy detected.")
                errors.append("  - Review welding process parameters and equipment calibration.")

        if 'Cell_Voltage' in self.data.columns and 'Cell_Capacity' in self.data.columns:
            energy_density = self.data['Cell_Voltage'] * self.data['Cell_Capacity']
            if (energy_density < 7000).any() or (energy_density > 25000).any():
                errors.append("Unusual energy density detected in some cells.")
                errors.append("  - Verify cell specifications and quality control processes.")

        return errors

    def run_analysis(self):
        print("Generating and preprocessing data...")
        self.generate_data()
        X_train, X_test, y_defect_train, y_defect_test, y_efficiency_train, y_efficiency_test = self.preprocess_data()

        print("Training models...")
        self.train_models(X_train, y_defect_train, y_efficiency_train)

        print("Detecting anomalies...")
        anomalies = self.detect_anomalies(self.data)
        print(f"Detected {len(anomalies)} anomalies.")

        print("Analyzing and generating recommendations...")
        recommendations, feature_importance, defect_predictions, efficiency_predictions = self.analyze_and_recommend(X_test)

        print("Visualizing results...")
        self.visualize_results(feature_importance, y_defect_test, defect_predictions, y_efficiency_test, efficiency_predictions)

        print("\nRecommendations:")
        for rec in recommendations:
            print(f"- {rec}")

        print("\nDetecting errors and suggesting improvements...")
        errors = self.detect_errors_and_suggest()
        for error in errors:
            print(error)

        print("\nRunning a what-if scenario...")
        scenario_result = self.what_if_scenario({'Welding_Current': 250, 'Welding_Time': 0.7})
        print("What-if scenario result:", scenario_result)

        print("\nOptimizing welding process...")
        optimization_result = self.optimize_process('Welding_Current', constraints={'Welding_Time': 0.5})
        print("Optimization result:", optimization_result)

if __name__ == "__main__":
    analyzer = EVModuleAssemblyAnalyzer()
    analyzer.run_analysis()

[I 2024-09-19 16:57:51,169] A new study created in memory with name: no-name-b434e3d9-b24b-468c-8240-8d7426cdc304


Generating and preprocessing data...
Training models...


[I 2024-09-19 16:58:03,110] Trial 0 finished with value: -32.501881704439 and parameters: {'n_estimators': 92, 'max_depth': 9, 'min_samples_split': 6, 'min_samples_leaf': 1}. Best is trial 0 with value: -32.501881704439.
[I 2024-09-19 16:58:10,968] Trial 1 finished with value: -32.49948482736541 and parameters: {'n_estimators': 57, 'max_depth': 9, 'min_samples_split': 5, 'min_samples_leaf': 1}. Best is trial 1 with value: -32.49948482736541.
[I 2024-09-19 16:58:25,149] Trial 2 finished with value: -32.11917914657607 and parameters: {'n_estimators': 177, 'max_depth': 5, 'min_samples_split': 6, 'min_samples_leaf': 2}. Best is trial 2 with value: -32.11917914657607.
[I 2024-09-19 16:58:47,681] Trial 3 finished with value: -32.440848640125296 and parameters: {'n_estimators': 163, 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 1}. Best is trial 2 with value: -32.11917914657607.
[I 2024-09-19 16:58:55,033] Trial 4 finished with value: -32.37003781249895 and parameters: {'n_esti

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000980 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3137
[LightGBM] [Info] Number of data points in the train set: 2666, number of used features: 29
[LightGBM] [Info] Start training from score -3.013137
[LightGBM] [Info] Start training from score -2.275206
[LightGBM] [Info] Start training from score -1.964079
[LightGBM] [Info] Start training from score -0.345591
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000981 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3137
[LightGBM] [Info] Number of data points in the train set: 2667, number of used features: 29
[LightGBM] [Info] Start training from score -3.013512
[LightGBM] [Info] Start training from score -2.275581
[LightGBM] [Info] Start training fro

[I 2024-09-19 17:04:20,696] Trial 0 finished with value: -31.941460552498967 and parameters: {'num_leaves': 42, 'learning_rate': 0.0014796764555403258, 'n_estimators': 140, 'min_child_samples': 15, 'subsample': 0.7882966172360444}. Best is trial 0 with value: -31.941460552498967.
<ipython-input-5-603f2f176d0a>:60: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.5),  # Reduced upper bound from 1.0 to 0.5
<ipython-input-5-603f2f176d0a>:63: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.6, 1.0)


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


[I 2024-09-19 17:10:41,234] Trial 18 finished with value: 289.71084569205107 and parameters: {'Welding_Current': 132.49642943266355}. Best is trial 12 with value: 290.52121933730973.
<ipython-input-5-603f2f176d0a>:302: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


[I 2024-09-19 17:10:41,380] Trial 19 finished with value: 290.5101338270558 and parameters: {'Welding_Current': 102.34054030874594}. Best is trial 12 with value: 290.52121933730973.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Optimization result: {'optimal_value': 101.88260091463856, 'predicted_outcome': {'defect_probability': {'No_Defect': 3, 'Minor_Defect': 0, 'Major_Defect': 0, 'Critical_Defect': 0}, 'efficiency_score': 90.52121933730973}}


# Intializing EVModuleAssemblyAnalyzer Class

This code initializes an EVModuleAssemblyAnalyzer, generates and preprocesses data, trains models, and then sets up a Flask web application with various routes for model performance analysis, feature importance visualization, anomaly detection, what-if scenarios, process optimization, and error detection. The app is configured to run on a random port and provide a public URL for access.

In [ ]:
analyzer = EVModuleAssemblyAnalyzer()

print("Generating and preprocessing data...")
analyzer.generate_data()
X_train, X_test, y_defect_train, y_defect_test, y_efficiency_train, y_efficiency_test = analyzer.preprocess_data()

print("Training models...")
analyzer.train_models(X_train, y_defect_train, y_efficiency_train)

print("Model training complete!")

app = Flask(__name__)
CORS(app)

@app.route('/model_performance', methods=['GET'])
def model_performance():
    y_pred_defect, y_pred_efficiency = analyzer.ensemble_predict(X_test)

    defect_accuracy = accuracy_score(y_defect_test, y_pred_defect)
    efficiency_r2 = r2_score(y_efficiency_test, y_pred_efficiency)
    efficiency_mse = mean_squared_error(y_efficiency_test, y_pred_efficiency)

    return jsonify({
        'defect_accuracy': defect_accuracy,
        'efficiency_r2': efficiency_r2,
        'efficiency_mse': efficiency_mse
    })

@app.route('/feature_importance', methods=['GET'])
def feature_importance():
    feature_importance = analyzer.models['rf_defect'].feature_importances_
    sorted_idx = np.argsort(feature_importance)
    top_features = analyzer.feature_names[sorted_idx[-10:]]
    top_importance = feature_importance[sorted_idx[-10:]]

    fig = px.bar(x=top_importance, y=top_features, orientation='h', title='Top 10 Feature Importance')
    return json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)

@app.route('/confusion_matrix', methods=['GET'])
def confusion_matrix_data():
    y_pred_defect, _ = analyzer.ensemble_predict(X_test)
    cm = confusion_matrix(y_defect_test, y_pred_defect)
    defect_labels = analyzer.label_encoder.classes_

    fig = px.imshow(cm, labels=dict(x="Predicted", y="Actual"), x=defect_labels, y=defect_labels,
                    title='Confusion Matrix - Defect Prediction', color_continuous_scale='Blues')
    fig.update_xaxes(side="top")
    return json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)

@app.route('/efficiency_scatter', methods=['GET'])
def efficiency_scatter():
    _, y_pred_efficiency = analyzer.ensemble_predict(X_test)

    fig = px.scatter(x=y_efficiency_test, y=y_pred_efficiency,
                     labels={'x': 'Actual Efficiency', 'y': 'Predicted Efficiency'},
                     title='Efficiency Prediction Performance')
    fig.add_trace(go.Scatter(x=[min(y_efficiency_test), max(y_efficiency_test)],
                             y=[min(y_efficiency_test), max(y_efficiency_test)],
                             mode='lines', name='Perfect Prediction'))
    return json.dumps(fig, cls=plotly.utils.PlotlyJSONEncoder)

@app.route('/anomalies', methods=['GET'])
def anomaly_detection():
    anomalies = analyzer.detect_anomalies(analyzer.data)
    return jsonify({
        'num_anomalies': len(anomalies),
        'anomaly_percentage': (len(anomalies) / len(analyzer.data)) * 100
    })

@app.route('/what_if', methods=['POST'])
def what_if_scenario():
    data = request.json

    # Map the input keys to the ones expected by the model
    input_mapping = {
        'cellVoltage': 'Cell_Voltage',
        'cellImpedance': 'Cell_Impedance',
        'cellCapacity': 'Cell_Capacity',
        'compressionForce': 'Compression_Force',
        'weldingCurrent': 'Welding_Current',
        'weldingTime': 'Welding_Time',
        'torque': 'Torque',
        'assemblyTime': 'Assembly_Time',
        'leakageRate': 'Leakage_Rate'
    }

    # Create the scenario dictionary with the correct keys
    scenario = {input_mapping[key]: float(value) for key, value in data.items() if value}

    # Run the what-if scenario
    result = analyzer.what_if_scenario(scenario)

    # Format the result for the front-end
    formatted_result = {
        'defectProbability': {k: float(v) for k, v in result['defect_probability'].items()},
        'efficiencyScore': float(result['efficiency_score'])
    }

    return jsonify(formatted_result)

@app.route('/optimize', methods=['POST'])
def optimize_process():
    data = request.json
    target_feature = data.get('targetFeature')
    constraints = data.get('constraints', {})

    # Run the optimization
    result = analyzer.optimize_process(target_feature, constraints)

    # Format the result for the front-end
    formatted_result = {
        'optimalValue': float(result['optimal_value']),
        'predictedOutcome': {
            'defectProbability': {k: float(v) for k, v in result['predicted_outcome']['defect_probability'].items()},
            'efficiencyScore': float(result['predicted_outcome']['efficiency_score'])
        }
    }

    return jsonify(formatted_result)

@app.route('/error_detection', methods=['GET'])
def error_detection():
    errors = analyzer.detect_errors_and_suggest()
    return jsonify({'errors': errors})

def get_public_url(port):
    return eval_js(f"google.colab.kernel.proxyPort({port})")

if __name__ == '__main__':
    port = random.randint(5000, 9000)  # Random port number
    public_url = get_public_url(port)
    print(f"Public URL: {public_url}")
    app.run(port=port, host='0.0.0.0')


[I 2024-09-19 17:24:49,576] A new study created in memory with name: no-name-b33829b1-7b82-4f55-9340-5c06c7fe4160


Generating and preprocessing data...
Training models...


[I 2024-09-19 17:24:55,720] Trial 0 finished with value: -32.311645510899325 and parameters: {'n_estimators': 63, 'max_depth': 7, 'min_samples_split': 5, 'min_samples_leaf': 4}. Best is trial 0 with value: -32.311645510899325.
[I 2024-09-19 17:25:12,035] Trial 1 finished with value: -32.42511313415713 and parameters: {'n_estimators': 125, 'max_depth': 9, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 0 with value: -32.311645510899325.
[I 2024-09-19 17:25:16,859] Trial 2 finished with value: -32.02350218556482 and parameters: {'n_estimators': 80, 'max_depth': 3, 'min_samples_split': 2, 'min_samples_leaf': 5}. Best is trial 2 with value: -32.02350218556482.
[I 2024-09-19 17:25:28,568] Trial 3 finished with value: -32.122696671680224 and parameters: {'n_estimators': 145, 'max_depth': 5, 'min_samples_split': 3, 'min_samples_leaf': 3}. Best is trial 2 with value: -32.02350218556482.
[I 2024-09-19 17:25:47,232] Trial 4 finished with value: -32.49219354108488 and parameters: {'

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000537 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3137
[LightGBM] [Info] Number of data points in the train set: 2666, number of used features: 29
[LightGBM] [Info] Start training from score -3.013137
[LightGBM] [Info] Start training from score -2.275206
[LightGBM] [Info] Start training from score -1.964079
[LightGBM] [Info] Start training from score -0.345591
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000469 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3137
[LightGBM] [Info] Number of data points in the train set: 2667, number of used features: 29
[LightGBM] [Info] Start training from score -3.013512
[LightGBM] [Info] Start tr

[I 2024-09-19 17:31:34,898] Trial 0 finished with value: -37.98534844919467 and parameters: {'num_leaves': 21, 'learning_rate': 0.2414331870872899, 'n_estimators': 191, 'min_child_samples': 5, 'subsample': 0.7291668519503334}. Best is trial 0 with value: -37.98534844919467.
<ipython-input-5-603f2f176d0a>:60: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

<ipython-input-5-603f2f176d0a>:63: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.



...

[I 2024-09-19 17:33:08,386] Trial 19 finished with value: -32.14365340727846 and parameters: {'num_leaves': 24, 'learning_rate': 0.007408019514295731, 'n_estimators': 102, 'min_child_samples': 13, 'subsample': 0.8625049834718034}. Best is trial 18 with value: -31.868924619677074.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000466 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3137
[LightGBM] [Info] Number of data points in the train set: 2667, number of used features: 29
[LightGBM] [Info] Start training from score 90.161450
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000814 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3137
[LightGBM] [Info] Number of data points in the train set: 4000, number of used features: 29
[LightGBM] [Info] Start training from score -3.015935
[LightGBM] [Info] Start training from score -2.275456
[LightGBM] [Info] Start training from score -1.962548
[LightGBM] [Info] Start training from score -0.345664
[LightGBM] [Info] Auto-cho

[I 2024-09-19 17:33:09,681] A new study created in memory with name: no-name-debc50c2-9fb9-4ae9-b73d-bbaea5626ea8
<ipython-input-5-603f2f176d0a>:70: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

[I 2024-09-19 17:33:18,236] Trial 0 finished with value: -32.58888889316658 and parameters: {'depth': 7, 'learning_rate': 0.07972829148650595, 'iterations': 70}. Best is trial 0 with value: -32.58888889316658.
<ipython-input-5-603f2f176d0a>:70: FutureWarning:

suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.

[I 2024-09-19 17:33:24,886] Trial 1 finished with value: -40.58143934213201 and parameters: {'depth': 5, 'learning_rate': 0.46264534560142007, 'iterations': 191}. Best is trial 0 with value: -

Model training complete!
Public URL: https://ccz88tuk3wv-496ff2e9c6d22116-6085-colab.googleusercontent.com/
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:6085
 * Running on http://172.28.0.12:6085
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [19/Sep/2024 17:43:08] "GET /?authuser=0 HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Sep/2024 18:15:17] "GET /?authuser=0 HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Sep/2024 18:15:22] "GET /data?authuser=0 HTTP/1.1" 404 -
